In [6]:
pip install python-bitcoinrpc

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException

# Replace these with your actual RPC credentials from bitcoin.conf
rpc_user = "nidarsana02"
rpc_password = "hjsHjZR1OIAft7Y-lYScC9F94PR54eAvp0vGPurIY1w"
rpc_host = "127.0.0.1"
rpc_port = 18443

# Create an RPC connection
rpc_client = AuthServiceProxy(f"http://{rpc_user}:{rpc_password}@{rpc_host}:{rpc_port}", timeout=120)

# Function to ensure wallet is loaded or created
def ensure_wallet_loaded(wallet_name):
    try:
        # Check if wallet is already loaded
        loaded_wallets = rpc_client.listwallets()
        if wallet_name in loaded_wallets:
            print(f"✅ Wallet '{wallet_name}' is already loaded.")
            return

        # Try loading the wallet
        print(f"🔄 Attempting to load wallet '{wallet_name}'...")
        rpc_client.loadwallet(wallet_name)
        print(f"✅ Wallet '{wallet_name}' loaded successfully.")

    except JSONRPCException as e:
        if e.error.get("code") == -18:  # Wallet does not exist
            print(f"⚠️ Wallet '{wallet_name}' does not exist. Creating it...")
            rpc_client.createwallet(wallet_name)
            print(f"✅ Wallet '{wallet_name}' created and loaded successfully.")
        else:
            print(f"❌ Unexpected error: {e}")
            raise

# Ensure the wallet is loaded or created
wallet_name = "CS216"
ensure_wallet_loaded(wallet_name)

# Reconnect to the wallet after ensuring it is loaded
rpc_client = AuthServiceProxy(f"http://{rpc_user}:{rpc_password}@{rpc_host}:{rpc_port}/wallet/{wallet_name}")

# Generate legacy addresses
address_A = rpc_client.getnewaddress("A", "legacy")
address_B = rpc_client.getnewaddress("B", "legacy")
address_C = rpc_client.getnewaddress("C", "legacy")

print("Address A:", address_A)
print("Address B:", address_B)
print("Address C:", address_C)


✅ Wallet 'CS216' is already loaded.
Address A: mfzekGpAapypBJwj7ePKvntKKdx81qBgzW
Address B: mvW3y8oqnN4EhTsvxQvGYi1LXUKVBEBMNZ
Address C: mqoMjr9QUTiaUKXgm2q115fTPq5SnXNrSy


In [8]:
# Fund address A
rpc_client.sendtoaddress(address_A, 0.01)


'e4aa9c5e454b9ede9a0dcafbafd02a5c68c6997d6e2d32498b7cb474ef006a25'

In [9]:
from decimal import Decimal

# Get the unspent outputs for address A
unspent_outputs = rpc_client.listunspent(0, 9999999, [address_A])

# Select the first unspent output
utxo = unspent_outputs[0]

# Create a raw transaction
raw_tx = rpc_client.createrawtransaction(
    [{"txid": utxo["txid"], "vout": utxo["vout"]}],
    {address_B: utxo["amount"] - Decimal("0.0001")}
)

# Decode the raw transaction
decoded_raw_tx = rpc_client.decoderawtransaction(raw_tx)
print(decoded_raw_tx)

# Sign the transaction using the wallet
signed_tx = rpc_client.signrawtransactionwithwallet(raw_tx)

# Decode the signed transaction
decoded_signed_tx = rpc_client.decoderawtransaction(signed_tx['hex'])
print("\nSigned Transaction from A to B:")
print(decoded_signed_tx)
signed_tx_hex = signed_tx['hex']

# Test mempool acceptance
test_result = rpc_client.testmempoolaccept([signed_tx_hex])

# Check if transaction is valid
if test_result[0]['allowed']:
    print("✅ Transaction is valid and will be accepted by the mempool.")
else:
    print(f"❌ Transaction is invalid. Reason: {test_result[0]['reject-reason']}")

# Broadcast the transaction
txid_A_to_B = rpc_client.sendrawtransaction(signed_tx['hex'])
print("Transaction ID from A to B:", txid_A_to_B)


{'txid': 'c5fe4abe021a9bcc842c7230666fe86eaab4f029e90881200a3cf23536aa40f7', 'hash': 'c5fe4abe021a9bcc842c7230666fe86eaab4f029e90881200a3cf23536aa40f7', 'version': 2, 'size': 85, 'vsize': 85, 'weight': 340, 'locktime': 0, 'vin': [{'txid': 'e4aa9c5e454b9ede9a0dcafbafd02a5c68c6997d6e2d32498b7cb474ef006a25', 'vout': 1, 'scriptSig': {'asm': '', 'hex': ''}, 'sequence': 4294967293}], 'vout': [{'value': Decimal('0.00990000'), 'n': 0, 'scriptPubKey': {'asm': 'OP_DUP OP_HASH160 a45e3501d3b88500f96b646666d9eb927c81a3a6 OP_EQUALVERIFY OP_CHECKSIG', 'desc': 'addr(mvW3y8oqnN4EhTsvxQvGYi1LXUKVBEBMNZ)#u93mtl8q', 'hex': '76a914a45e3501d3b88500f96b646666d9eb927c81a3a688ac', 'address': 'mvW3y8oqnN4EhTsvxQvGYi1LXUKVBEBMNZ', 'type': 'pubkeyhash'}}]}

Signed Transaction from A to B:
{'txid': 'fbb76e569367839be149d17d465800ca550893a114eb67ba29f05ccb4e9c0d0a', 'hash': 'fbb76e569367839be149d17d465800ca550893a114eb67ba29f05ccb4e9c0d0a', 'version': 2, 'size': 191, 'vsize': 191, 'weight': 764, 'locktime': 0, 'vi

In [10]:
# Get the unspent outputs for address B
unspent_outputs_B = rpc_client.listunspent(0, 9999999, [address_B])

# Ensure there are UTXOs available
if not unspent_outputs_B:
    raise ValueError("No UTXOs available for address B!")

# Select the first unspent output (UTXO from A → B)
utxo_B = unspent_outputs_B[0]
txid_A_to_B = utxo_B["txid"]

# Retrieve the transaction A → B to extract scriptPubKey
tx_A_to_B = rpc_client.getrawtransaction(txid_A_to_B, True)
scriptPubKey_hex_A = tx_A_to_B["vout"][utxo_B["vout"]]["scriptPubKey"]["hex"]

# Create a raw transaction from B to C
fee = Decimal("0.0001")
raw_tx_B = rpc_client.createrawtransaction(
    [{"txid": utxo_B["txid"], "vout": utxo_B["vout"]}],
    {address_C: utxo_B["amount"] - fee}
)

# Decode the raw transaction
decoded_raw_tx_B = rpc_client.decoderawtransaction(raw_tx_B)
print("Decoded Raw Transaction from B to C:", decoded_raw_tx_B)

# Sign the transaction using the wallet
signed_tx_B = rpc_client.signrawtransactionwithwallet(raw_tx_B)

# Decode the signed transaction
decoded_signed_tx_B = rpc_client.decoderawtransaction(signed_tx_B['hex'])
print("\nSigned Transaction from B to C:")
print(decoded_signed_tx_B)

# Extract scriptSig from B → C (unlocking A's UTXO)
scriptSig_asm_B = decoded_signed_tx_B['vin'][0].get('scriptSig', {}).get('asm', "Missing ScriptSig")

print("\n🔍 Comparing Scripts:")
print(f"scriptSig_asm_B (B unlocking A's UTXO): {scriptSig_asm_B}")
print(f"scriptPubKey_hex_A (Locking script from A → B): {scriptPubKey_hex_A}")



Decoded Raw Transaction from B to C: {'txid': '889e6cc1c2c613541d0b6016111f419f786a3ad2f50b6321d5142a535cb6a34d', 'hash': '889e6cc1c2c613541d0b6016111f419f786a3ad2f50b6321d5142a535cb6a34d', 'version': 2, 'size': 85, 'vsize': 85, 'weight': 340, 'locktime': 0, 'vin': [{'txid': 'fbb76e569367839be149d17d465800ca550893a114eb67ba29f05ccb4e9c0d0a', 'vout': 0, 'scriptSig': {'asm': '', 'hex': ''}, 'sequence': 4294967293}], 'vout': [{'value': Decimal('0.00980000'), 'n': 0, 'scriptPubKey': {'asm': 'OP_DUP OP_HASH160 70cb7be0c23190582abe9b0a0aa66c6b3b98d606 OP_EQUALVERIFY OP_CHECKSIG', 'desc': 'addr(mqoMjr9QUTiaUKXgm2q115fTPq5SnXNrSy)#vqzn0jza', 'hex': '76a91470cb7be0c23190582abe9b0a0aa66c6b3b98d60688ac', 'address': 'mqoMjr9QUTiaUKXgm2q115fTPq5SnXNrSy', 'type': 'pubkeyhash'}}]}

Signed Transaction from B to C:
{'txid': 'be65bf8a4b602c8ed08da9666c7829a1b6bea56a3c891b9e279c4bc4fee5b016', 'hash': 'be65bf8a4b602c8ed08da9666c7829a1b6bea56a3c891b9e279c4bc4fee5b016', 'version': 2, 'size': 191, 'vsize': 1